<a href="https://colab.research.google.com/github/ClaudeanSoares/santander-dev-week-etl/blob/main/pipeline_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==============================================================================
# 🚀 SANTANDER DEV WEEK 2025: PIPELINE ETL COM IA GENERATIVA
# Autor: Claudean Soares
# Objetivo: Personalizar o marketing bancário usando Python e OpenAI
# ==============================================================================

# [0] INSTALAÇÃO E IMPORTAÇÃO
!pip install pandas requests openai -q

import pandas as pd
import requests
import openai
import json
from google.colab import userdata

# Configurações de API (Garanta que a chave está nos 'Secrets' do Colab)
openai.api_key = userdata.get('OPENAI_API_KEY')
SDW_API_URL = 'https://sdw-2023-prd.up.railway.app'

# --- [PASSO 1: SETUP & EXTRACT] ---
def setup_and_extract():
    print("📂 Criando base de IDs e extraindo dados da API...")
    # Criando o arquivo CSV para evitar o FileNotFoundError
    df_ids = pd.DataFrame({'UserID': [3827, 3831, 3832]})
    df_ids.to_csv('SDW2023.csv', index=False)

    # Lendo os IDs e buscando na API
    user_ids = pd.read_csv('SDW2023.csv')['UserID'].tolist()
    users = []
    for id in user_ids:
        response = requests.get(f'{SDW_API_URL}/users/{id}')
        if response.status_code == 200:
            users.append(response.json())
    print(f"✅ {len(users)} clientes extraídos com sucesso.")
    return users

# --- [PASSO 2: TRANSFORM] ---
def transform_with_ai(users):
    print("🤖 Gerando mensagens personalizadas com GPT-3.5...")
    for user in users:
        # Chamada profissional para a OpenAI
        completion = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Você é um consultor financeiro do Santander."},
                {"role": "user", "content": f"Crie uma mensagem curta para {user['name']} sobre investimentos (máx 100 caracteres)"}
            ]
        )
        msg = completion.choices[0].message.content.strip('\"')

        # Adicionando ao objeto do usuário
        user['news'].append({
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
            "description": msg
        })
    print("✅ Transformação concluída.")
    return users

# --- [PASSO 3: LOAD] ---
def load_to_api(users):
    print("🚀 Carregando dados atualizados de volta para a API...")
    for user in users:
        response = requests.put(f"{SDW_API_URL}/users/{user['id']}", json=user)
        status = "Sucesso" if response.status_code == 200 else "Falha"
        print(f"🔹 Cliente {user['name']}: {status}")

# --- EXECUÇÃO FINAL ---
if __name__ == "__main__":
    lista_clientes = setup_and_extract()
    if lista_clientes:
        clientes_enriquecidos = transform_with_ai(lista_clientes)
        load_to_api(clientes_enriquecidos)
        print("\n🏆 Projeto Finalizado! Pronto para o GitHub.")




📂 Criando base de IDs e extraindo dados da API...
✅ 0 clientes extraídos com sucesso.


🚀 Santander Dev Week 2025 - Ciência de Dados
Autor: Claudean Soares

Descrição: Este projeto implementa um pipeline ETL (Extração, Transformação e Carga) integrando a API do Santander com a inteligência artificial da OpenAI para criar mensagens de marketing personalizadas.

Nota de Autoria: Projeto desenvolvido durante a Santander Dev Week 2025, com o auxílio de IA Generativa para a estruturação do pipeline ETL e resolução de bugs de ambiente local.

# Nova seção